In [19]:
pip install PyWavelets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import librosa
import parselmouth
from parselmouth.praat import call
import scipy
from scipy.stats import entropy
from scipy.signal import hilbert
import numpy as np
import pywt
#from tqwtpy import tqwt
import nolds
import pandas as pd

In [21]:
# Function to extract Pitch Period Entropy (PPE)
def extract_ppe(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = call(snd, "To Pitch", 0.0, 75, 600)  # Extract pitch
    
    # Extract pitch mean in Hertz
    ppe = call(pitch, "Get mean", 0, 0, "Hertz")
    return ppe

In [22]:
# Function to extract DFA (Detrended Fluctuation Analysis)
def extract_dfa(y):
    return nolds.dfa(y)

In [23]:
# Function to extract RPDE (Recurrence Period Density Entropy)
def extract_rpde(y):
    rpde_value = scipy.stats.entropy(np.abs(np.fft.rfft(y)))
    return rpde_value

In [24]:
# Function to extract jitter and shimmer features using Praat
def extract_jitter_shimmer(audio_path):
    snd = parselmouth.Sound(audio_path)
    point_process = call(snd, "To PointProcess (periodic, cc)", 75, 500)
    
    jitter_local = call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_rap = call(point_process, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ppq5 = call(point_process, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ddp = 3 * jitter_rap  # DDP = 3 * RAP jitter
    
    shimmer_local = call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_db = call([snd, point_process], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq3 = call([snd, point_process], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq5 = call([snd, point_process], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq11 = call([snd, point_process], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_dda = 3 * shimmer_apq3  # DDA = 3 * APQ3 shimmer
    
    return (
        jitter_local, jitter_rap, jitter_ppq5, jitter_ddp, shimmer_local, shimmer_db, 
        shimmer_apq3, shimmer_apq5, shimmer_apq11, shimmer_dda
    )

In [25]:
# Function to extract Harmonicity-related features
def extract_harmonicity_full(audio_path):
    snd = parselmouth.Sound(audio_path)

    # Harmonicity (cc) for mean harmonicity
    harmonicity_cc = call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    mean_harmonicity = call(harmonicity_cc, "Get mean", 0, 0)

    return mean_harmonicity

In [26]:
# Function to extract pulse-related features using Praat (corrected)
def extract_pulse_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    point_process = call(snd, "To PointProcess (periodic, cc)", 75, 500)
    
    num_pulses = call(point_process, "Get number of points")  # Get pulse count
    
    # Get times of each pulse and calculate the intervals (periods)
    pulses_times = [call(point_process, "Get time from index", i + 1) for i in range(num_pulses)]
    periods = np.diff(pulses_times)  # Differences between consecutive pulses
    
    num_periods_pulses = len(periods)
    mean_period_pulses = np.mean(periods) if len(periods) > 0 else 0
    stddev_period_pulses = np.std(periods) if len(periods) > 0 else 0
    
    return num_pulses, num_periods_pulses, mean_period_pulses, stddev_period_pulses


In [27]:
def extract_intensity_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # Convert sound to intensity
    intensity = call(snd, "To Intensity", 75, 0, "yes")
    
    # Extract intensity features
    min_intensity = call(intensity, "Get minimum", 0, 0, "Parabolic")
    max_intensity = call(intensity, "Get maximum", 0, 0, "Parabolic")
    mean_intensity = call(intensity, "Get mean", 0, 0)

    return min_intensity, max_intensity, mean_intensity

In [28]:
def extract_formant_frequencies(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # Create a formant object
    formant = call(snd, "To Formant (burg)", 0.025, 5, 5500, 0.02, 50)

    # Extract the formant frequencies in Hertz
    f1 = call(formant, "Get mean", 1, 0, 0, "Hertz")  # 1st formant
    f2 = call(formant, "Get mean", 2, 0, 0, "Hertz")  # 2nd formant
    f3 = call(formant, "Get mean", 3, 0, 0, "Hertz")  # 3rd formant
    f4 = call(formant, "Get mean", 4, 0, 0, "Hertz")  # 4th formant

    return f1, f2, f3, f4


In [29]:
def extract_formant_bandwidths(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # Create a formant object
    formant = call(snd, "To Formant (burg)", 0.025, 5, 5500, 0.02, 50)

    # Extract the bandwidths for the first four formants with interpolation
    b1 = call(formant, "Get bandwidth at time", 1, 0.5, "Hertz", "linear")  # 1st formant bandwidth
    b2 = call(formant, "Get bandwidth at time", 2, 0.5, "Hertz", "linear")  # 2nd formant bandwidth
    b3 = call(formant, "Get bandwidth at time", 3, 0.5, "Hertz", "linear")  # 3rd formant bandwidth
    b4 = call(formant, "Get bandwidth at time", 4, 0.5, "Hertz", "linear")  # 4th formant bandwidth

    return b1, b2, b3, b4

In [30]:
def extract_vocal_fold_features(audio_path):
    # Load audio file using librosa
    y, sr = librosa.load(audio_path, sr=None)

    # Placeholder logic: actual computation of each feature can vary based on exact method
    GQ_prc5_95 = np.percentile(y, [5, 95])
    GQ_std_cycle_open = np.std(y[:len(y)//2])  # Placeholder for cycle open
    GQ_std_cycle_closed = np.std(y[len(y)//2:])  # Placeholder for cycle closed
    GNE_mean = np.mean(y)
    GNE_std = np.std(y)
    
    # SNR and NSR calculations (using available librosa features)
    GNE_SNR_TKEO = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    GNE_SNR_SEO = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    GNE_NSR_TKEO = np.mean(librosa.feature.zero_crossing_rate(y))
    GNE_NSR_SEO = np.mean(librosa.feature.rms(y=y))

    VFER_mean = np.mean(y)
    VFER_std = np.std(y)
    VFER_entropy = -np.sum(y * np.log(np.abs(y) + 1e-9))  # Entropy calculation (abs added for safety)
    VFER_SNR_TKEO = np.std(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    VFER_SNR_SEO = np.std(librosa.feature.spectral_rolloff(y=y, sr=sr))
    VFER_NSR_TKEO = np.std(librosa.feature.zero_crossing_rate(y))
    VFER_NSR_SEO = np.std(librosa.feature.rms(y=y))

    IMF_SNR_SEO = np.std(librosa.feature.spectral_rolloff(y=y, sr=sr))  # Placeholder
    IMF_SNR_TKEO = np.std(librosa.feature.spectral_bandwidth(y=y, sr=sr))  # Placeholder
    IMF_SNR_entropy = -np.sum(librosa.feature.spectral_bandwidth(y=y, sr=sr) * 
                              np.log(np.abs(librosa.feature.spectral_bandwidth(y=y, sr=sr)) + 1e-9))  # Entropy placeholder
    IMF_NSR_SEO = np.std(librosa.feature.zero_crossing_rate(y))
    IMF_NSR_TKEO = np.std(librosa.feature.zero_crossing_rate(y))
    IMF_NSR_entropy = -np.sum(librosa.feature.rms(y=y) * np.log(np.abs(librosa.feature.rms(y=y)) + 1e-9))  # Placeholder for entropy

    return {
        'GQ_prc5_95_min': GQ_prc5_95[0],
        'GQ_prc5_95_max': GQ_prc5_95[1],
        'GQ_std_cycle_open': GQ_std_cycle_open,
        'GQ_std_cycle_closed': GQ_std_cycle_closed,
        'GNE_mean': GNE_mean,
        'GNE_std': GNE_std,
        'GNE_SNR_TKEO': GNE_SNR_TKEO,
        'GNE_SNR_SEO': GNE_SNR_SEO,
        'GNE_NSR_TKEO': GNE_NSR_TKEO,
        'GNE_NSR_SEO': GNE_NSR_SEO,
        'VFER_mean': VFER_mean,
        'VFER_std': VFER_std,
        'VFER_entropy': VFER_entropy,
        'VFER_SNR_TKEO': VFER_SNR_TKEO,
        'VFER_SNR_SEO': VFER_SNR_SEO,
        'VFER_NSR_TKEO': VFER_NSR_TKEO,
        'VFER_NSR_SEO': VFER_NSR_SEO,
        'IMF_SNR_SEO': IMF_SNR_SEO,
        'IMF_SNR_TKEO': IMF_SNR_TKEO,
        'IMF_SNR_entropy': IMF_SNR_entropy,
        'IMF_NSR_SEO': IMF_NSR_SEO,
        'IMF_NSR_TKEO': IMF_NSR_TKEO,
        'IMF_NSR_entropy': IMF_NSR_entropy
    }


In [31]:
def extract_mfcc_features(audio_path):
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Compute MFCC (13 coefficients are standard, adjust if needed)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    # Compute deltas (first and second order differences)
    mfcc_deltas = librosa.feature.delta(mfccs)
    mfcc_delta_deltas = librosa.feature.delta(mfccs, order=2)

    # Compute log energy (square root of RMS energy)
    log_energy = np.log(np.mean(librosa.feature.rms(y=y)))

    # Collecting mean and standard deviation for MFCCs, deltas, and delta-deltas
    features = {
        'mean_Log_energy': log_energy,
        'mean_delta_log_energy': np.mean(mfcc_deltas),
        'mean_delta_delta_log_energy': np.mean(mfcc_delta_deltas),
        'std_Log_energy': np.std(log_energy),
        'std_delta_log_energy': np.std(mfcc_deltas),
        'std_delta_delta_log_energy': np.std(mfcc_delta_deltas)
    }

    # For each MFCC coefficient (0 to 12), add the mean and standard deviation of the MFCC, delta, and delta-delta
    for i in range(13):
        features[f'mean_MFCC_{i}th_coef'] = np.mean(mfccs[i, :])
        features[f'std_MFCC_{i}th_coef'] = np.std(mfccs[i, :])
        features[f'mean_{i}th_delta'] = np.mean(mfcc_deltas[i, :])
        features[f'std_{i}th_delta'] = np.std(mfcc_deltas[i, :])
        features[f'mean_{i}th_delta_delta'] = np.mean(mfcc_delta_deltas[i, :])
        features[f'std_{i}th_delta_delta'] = np.std(mfcc_delta_deltas[i, :])

    return features

In [32]:
# Helper function to compute Shannon entropy
def entropy_shannon(data):
    p_data = np.abs(data) / np.sum(np.abs(data))  # Normalize the data
    entropy = -np.sum(p_data * np.log2(p_data + 1e-12))  # Add small constant to avoid log(0)
    return entropy

# Helper function to compute log entropy
def entropy_log(data):
    return np.sum(np.log(np.abs(data) + 1e-12))

# Helper function to compute Teager-Kaiser Energy Operator (TKEO)
def compute_TKEO(data):
    TKEO = data[1:-1] ** 2 - data[:-2] * data[2:]
    return np.mean(TKEO), np.std(TKEO)


In [33]:
def extract_wavelet_features(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Perform wavelet decomposition (using 'db4' wavelet as an example)
    coeffs = pywt.wavedec(y, wavelet='db4', level=10)

    # Extract approximation (Ea) and detail (Ed) coefficients
    Ea = coeffs[0]  # Approximation
    Ed_coeffs = coeffs[1:]  # Detail coefficients

    # Initialize a dictionary to store wavelet features
    features = {}

    # Compute mean and standard deviation for approximation (Ea)
    features['Ea'] = np.mean(Ea)
    
    # Add statistics for each detail coefficient
    for i, Ed in enumerate(Ed_coeffs, 1):
        features[f'Ed_{i}_coef'] = np.mean(Ed)
        features[f'det_entropy_shannon_{i}_coef'] = entropy_shannon(Ed)
        features[f'det_entropy_log_{i}_coef'] = entropy_log(Ed)
        features[f'det_TKEO_mean_{i}_coef'], features[f'det_TKEO_std_{i}_coef'] = compute_TKEO(Ed)

    # Compute TKEO and entropy for the approximation (Ea) and add features
    features['app_TKEO_mean'], features['app_TKEO_std'] = compute_TKEO(Ea)
    features['app_entropy_shannon'] = entropy_shannon(Ea)
    features['app_entropy_log'] = entropy_log(Ea)

    return features

In [34]:
# Combine all extracted features
def extract_baseline_features(audio_path):
    # Load audio using librosa
    y, sr = librosa.load(audio_path, sr=None)
    
    # Extract PPE, DFA, RPDE features (assuming you have these functions)
    ppe = extract_ppe(audio_path)
    dfa_value = extract_dfa(y)
    rpde_value = extract_rpde(y)
    
    # Pulse-related features
    num_pulses, num_periods_pulses, mean_period_pulses, stddev_period_pulses = extract_pulse_features(audio_path)
    
    # Jitter and shimmer features
    jitter_local, jitter_rap, jitter_ppq5, jitter_ddp, shimmer_local, shimmer_db, shimmer_apq3, shimmer_apq5, shimmer_apq11, shimmer_dda = extract_jitter_shimmer(audio_path)
    
    # Harmonicity features (assuming you have this function)
    mean_harmonicity = extract_harmonicity_full(audio_path)

    # Intensity features
    min_intensity, max_intensity, mean_intensity = extract_intensity_features(audio_path)

    #Formant features
    f1,f2,f3,f4 = extract_formant_frequencies(audio_path)
    b1, b2, b3, b4 = extract_formant_bandwidths(audio_path)

    vocal_fold_features = extract_vocal_fold_features(audio_path)

    mfcc_features = extract_mfcc_features(audio_path)

    wavelet_features = extract_wavelet_features(audio_path)

    print(wavelet_features)
    
    # Combine all features into a single list
    features = [
        ppe, dfa_value, rpde_value,
        num_pulses, num_periods_pulses, mean_period_pulses, stddev_period_pulses,
        jitter_local, jitter_rap, jitter_ppq5, jitter_ddp,
        shimmer_local, shimmer_db, shimmer_apq3, shimmer_apq5, shimmer_apq11, shimmer_dda,
        mean_harmonicity,
        min_intensity, max_intensity, mean_intensity,
        f1,f2,f3,f4,
        b1, b2, b3, b4,
        vocal_fold_features['GQ_prc5_95_min'], vocal_fold_features['GQ_prc5_95_max'],
        vocal_fold_features['GQ_std_cycle_open'], vocal_fold_features['GQ_std_cycle_closed'],
        vocal_fold_features['GNE_mean'], vocal_fold_features['GNE_std'],
        vocal_fold_features['GNE_SNR_TKEO'], vocal_fold_features['GNE_SNR_SEO'],
        vocal_fold_features['GNE_NSR_TKEO'], vocal_fold_features['GNE_NSR_SEO'],
        vocal_fold_features['VFER_mean'], vocal_fold_features['VFER_std'],
        vocal_fold_features['VFER_entropy'], vocal_fold_features['VFER_SNR_TKEO'],
        vocal_fold_features['VFER_SNR_SEO'], vocal_fold_features['VFER_NSR_TKEO'],
        vocal_fold_features['VFER_NSR_SEO'], vocal_fold_features['IMF_SNR_SEO'],
        vocal_fold_features['IMF_SNR_TKEO'], vocal_fold_features['IMF_SNR_entropy'],
        vocal_fold_features['IMF_NSR_SEO'], vocal_fold_features['IMF_NSR_TKEO'],
        vocal_fold_features['IMF_NSR_entropy'],
    ]
    
    features += list(mfcc_features.values())

    features += list(wavelet_features.values())

    return features


In [35]:
def process_single_audio_file(audio_file, output_csv):
    columns = [
        'PPE', 'DFA', 'RPDE',
        'numPulses', 'numPeriodsPulses', 'meanPeriodPulses', 'stdDevPeriodPulses',
        'locPctJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
        'locShimmer', 'locDbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer',
        'meanAutoCorrHarmonicity',
        'min_intensity', 'max_intensity', 'mean_intensity',
        'f1','f2','f3','f4',
        'b1', 'b2', 'b3', 'b4',
        'GQ_prc5_95_low', 'GQ_prc5_95_high', 'GQ_std_cycle_open', 'GQ_std_cycle_closed',
        'GNE_mean', 'GNE_std', 'GNE_SNR_TKEO', 'GNE_SNR_SEO', 'GNE_NSR_TKEO', 'GNE_NSR_SEO',
        'VFER_mean', 'VFER_std', 'VFER_entropy', 'VFER_SNR_TKEO', 'VFER_SNR_SEO', 'VFER_NSR_TKEO', 'VFER_NSR_SEO',
        'IMF_SNR_SEO', 'IMF_SNR_TKEO', 'IMF_SNR_entropy', 'IMF_NSR_SEO', 'IMF_NSR_TKEO', 'IMF_NSR_entropy',
        'mean_Log_energy', 'mean_MFCC_0th_coef', 'mean_MFCC_1st_coef', 'mean_MFCC_2nd_coef', 
        'mean_MFCC_3rd_coef', 'mean_MFCC_4th_coef', 'mean_MFCC_5th_coef', 'mean_MFCC_6th_coef',
        'mean_MFCC_7th_coef', 'mean_MFCC_8th_coef', 'mean_MFCC_9th_coef', 'mean_MFCC_10th_coef', 
        'mean_MFCC_11th_coef', 'mean_MFCC_12th_coef',
        'mean_delta_log_energy', 'mean_0th_delta', 'mean_1st_delta', 'mean_2nd_delta', 'mean_3rd_delta', 
        'mean_4th_delta', 'mean_5th_delta', 'mean_6th_delta', 'mean_7th_delta', 'mean_8th_delta', 
        'mean_9th_delta', 'mean_10th_delta', 'mean_11th_delta', 'mean_12th_delta',
        'mean_delta_delta_log_energy', 'mean_delta_delta_0th', 'mean_1st_delta_delta', 'mean_2nd_delta_delta', 
        'mean_3rd_delta_delta', 'mean_4th_delta_delta', 'mean_5th_delta_delta', 'mean_6th_delta_delta', 
        'mean_7th_delta_delta', 'mean_8th_delta_delta', 'mean_9th_delta_delta', 'mean_10th_delta_delta', 
        'mean_11th_delta_delta', 'mean_12th_delta_delta',
        'std_Log_energy', 'std_MFCC_0th_coef', 'std_MFCC_1st_coef', 'std_MFCC_2nd_coef', 
        'std_MFCC_3rd_coef', 'std_MFCC_4th_coef', 'std_MFCC_5th_coef', 'std_MFCC_6th_coef',
        'std_MFCC_7th_coef', 'std_MFCC_8th_coef', 'std_MFCC_9th_coef', 'std_MFCC_10th_coef', 
        'std_MFCC_11th_coef', 'std_MFCC_12th_coef',
        'std_delta_log_energy', 'std_0th_delta', 'std_1st_delta', 'std_2nd_delta', 'std_3rd_delta', 
        'std_4th_delta', 'std_5th_delta', 'std_6th_delta', 'std_7th_delta', 'std_8th_delta', 
        'std_9th_delta', 'std_10th_delta', 'std_11th_delta', 'std_12th_delta',
        'std_delta_delta_log_energy', 'std_delta_delta_0th', 'std_1st_delta_delta', 'std_2nd_delta_delta', 
        'std_3rd_delta_delta', 'std_4th_delta_delta', 'std_5th_delta_delta', 'std_6th_delta_delta', 
        'std_7th_delta_delta', 'std_8th_delta_delta', 'std_9th_delta_delta', 'std_10th_delta_delta', 
        'std_11th_delta_delta', 'std_12th_delta_delta',
        'Ea',
        'Ed_1_coef', 'det_entropy_shannon_1_coef', 'det_entropy_log_1_coef', 'det_TKEO_mean_1_coef', 'det_TKEO_std_1_coef',
        'Ed_2_coef', 'det_entropy_shannon_2_coef', 'det_entropy_log_2_coef', 'det_TKEO_mean_2_coef', 'det_TKEO_std_2_coef',
        'Ed_3_coef', 'det_entropy_shannon_3_coef', 'det_entropy_log_3_coef', 'det_TKEO_mean_3_coef', 'det_TKEO_std_3_coef',
        'Ed_4_coef', 'det_entropy_shannon_4_coef', 'det_entropy_log_4_coef', 'det_TKEO_mean_4_coef', 'det_TKEO_std_4_coef',
        'Ed_5_coef', 'det_entropy_shannon_5_coef', 'det_entropy_log_5_coef', 'det_TKEO_mean_5_coef', 'det_TKEO_std_5_coef',
        'Ed_6_coef', 'det_entropy_shannon_6_coef', 'det_entropy_log_6_coef', 'det_TKEO_mean_6_coef', 'det_TKEO_std_6_coef',
        'Ed_7_coef', 'det_entropy_shannon_7_coef', 'det_entropy_log_7_coef', 'det_TKEO_mean_7_coef', 'det_TKEO_std_7_coef',
        'Ed_8_coef', 'det_entropy_shannon_8_coef', 'det_entropy_log_8_coef', 'det_TKEO_mean_8_coef', 'det_TKEO_std_8_coef',
        'Ed_9_coef', 'det_entropy_shannon_9_coef', 'det_entropy_log_9_coef', 'det_TKEO_mean_9_coef', 'det_TKEO_std_9_coef',
        'Ed_10_coef', 'det_entropy_shannon_10_coef', 'det_entropy_log_10_coef', 'det_TKEO_mean_10_coef', 'det_TKEO_std_10_coef',
        'app_TKEO_mean', 'app_TKEO_std', 'app_entropy_shannon', 'app_entropy_log'
    ]
    
    # Extract features for the single file
    baseline_features = extract_baseline_features(audio_file)
    
    # Convert to DataFrame
    df = pd.DataFrame([baseline_features], columns=columns)
    
    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"Features for {audio_file} saved to {output_csv}")

In [36]:
audio_file = '../data/raw/test.wav'  # Replace with your actual file path
output_csv = 'single_audio_features.csv'
process_single_audio_file(audio_file, output_csv)

{'Ea': -0.0013261768, 'Ed_1_coef': -0.00012896479, 'det_entropy_shannon_1_coef': 6.153825, 'det_entropy_log_1_coef': -521.0039, 'det_TKEO_mean_1_coef': 0.00016324293, 'det_TKEO_std_1_coef': 0.00013513498, 'Ed_2_coef': 0.00022400051, 'det_entropy_shannon_2_coef': 6.8003163, 'det_entropy_log_2_coef': -989.1493, 'det_TKEO_mean_2_coef': 0.00042453778, 'det_TKEO_std_2_coef': 0.0005353654, 'Ed_3_coef': 0.000111120025, 'det_entropy_shannon_3_coef': 7.6123714, 'det_entropy_log_3_coef': -1729.873, 'det_TKEO_mean_3_coef': 0.004261447, 'det_TKEO_std_3_coef': 0.0044377344, 'Ed_4_coef': 0.002310781, 'det_entropy_shannon_4_coef': 8.38087, 'det_entropy_log_4_coef': -3826.5222, 'det_TKEO_mean_4_coef': 0.0015555515, 'det_TKEO_std_4_coef': 0.0033536346, 'Ed_5_coef': -0.00073652505, 'det_entropy_shannon_5_coef': 9.210442, 'det_entropy_log_5_coef': -8048.52, 'det_TKEO_mean_5_coef': 0.0014134622, 'det_TKEO_std_5_coef': 0.0046957424, 'Ed_6_coef': -3.4502908e-05, 'det_entropy_shannon_6_coef': 10.119471, 'det

In [37]:
df = pd.read_csv("single_audio_features.csv")

In [38]:
df.head()

,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,rapJitter,ppq5Jitter,...,det_TKEO_std_9_coef,Ed_10_coef,det_entropy_shannon_10_coef,det_entropy_log_10_coef,det_TKEO_mean_10_coef,det_TKEO_std_10_coef,app_TKEO_mean,app_TKEO_std,app_entropy_shannon,app_entropy_log
0,128.200229,0.626037,8.74539,136,135,0.007927,0.001871,0.00845,0.003708,0.004168,...,0.000004,-1.129350e-08,14.529393,-464815.25,4.378862e-08,2.953155e-07,0.001929,0.003333,5.753789,-454.3848
